In [1]:
from datasets import load_from_disk
from pathlib import Path
from functools import partial
import os
import pandas as pd
from tqdm import tqdm

/home/nico/Thesis/neural-artwork-caption-generator/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
PROCESSED_DATA_DIR = Path.cwd().parent / "data" / "processed"
RAW_DATA_DIR = Path.cwd().parent / "data" / "raw"

In [3]:
dataset = load_from_disk(PROCESSED_DATA_DIR / "captioning_dataset_augmented_processed")

In [4]:
dataset = dataset["test"]

In [5]:
df = pd.read_csv(RAW_DATA_DIR / "artgraph_captions.csv")

In [6]:
def is_in_human_set(example):
    return df[df["image"] == example["file_name"]]["human"].values[0] == "y"

In [7]:
dataset = dataset.filter(is_in_human_set)
dataset.save_to_disk(PROCESSED_DATA_DIR / "captioning_dataset_augmented_test_human")

KeyboardInterrupt: 

In [ ]:
dataset

In [ ]:
def _extend_examples(examples):
    new_examples = dict((k, []) for k in examples.keys() if k != "captions")
    new_examples["caption"] = []

    for i in range(len(examples["image"])):
        for caption in examples["captions"][i]:
            for k in new_examples.keys():
                if k != "caption":
                    new_examples[k].append(examples[k][i])
                else:
                    new_examples[k].append(caption)

    return new_examples

dataset["train"] = dataset["train"].map(_extend_examples, batched=True, num_proc=4, remove_columns=["captions"])

In [ ]:
dataset["train"] = dataset["train"].shuffle(seed=42)
dataset["train"] = dataset["train"].flatten_indices()
dataset.save_to_disk(PROCESSED_DATA_DIR / "captioning_dataset_augmented_prepared")

In [ ]:
dataset["train"][0]["image"]